## 1 Introduction

Until now we have interacted with our single node cluster using the kubectl commands, but is usefull lear how to interact with the cluster using python. Now we will interact with the kubernetes cluster usign python pykube-ng library.

Let's start making some importations to use:
* the pykube-ng to interact with the cluster.
* json and yaml libs to better visualize the objects in this notebook.

In [3]:
import pykube
import json
import yaml

Now we can make use of the pykube lib with few steps like showed in README.
1. First lets create an api object to point to our cluster api.
2. the we will query a list of the runnig pods in the cluster.

In [4]:
#replace the string parameter with the name of your kubernetes config file.
api = pykube.HTTPClient(pykube.KubeConfig.from_file("k3s.yaml"))
#now we can use the pykube and the api object to get the list of pods
pods = pykube.Pod.objects(api)
#notice that the pods list is not a python list
print("type of object of pods list:",type(pods),"\n")
#but we can still iterate over the object to print information about the pod
for pod in pods:
    print(pod)

type of object of pods list: <class 'pykube.query.Query'> 

svclb-notebook-h7pxg
notebook-7dd7db75cd-g8h7f


Let's understand the object that we have iterated over.

In [6]:
#Replace the string parameter with one of the pods names printed by previous session
pod = pods.get_by_name("notebook-7dd7db75cd-g8h7f")
print(type(pod),"\n")
#we can get the properties inside the Pod object using the dir() function
print("This is the map of the pod object propeties:",dir(pod),"\n")

<class 'pykube.objects.Pod'> 

This is the map of the pod object propeties: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_original_obj', 'annotations', 'api', 'api_kwargs', 'base', 'create', 'delete', 'endpoint', 'exists', 'kind', 'labels', 'logs', 'metadata', 'name', 'namespace', 'obj', 'objects', 'patch', 'ready', 'reload', 'set_obj', 'update', 'version', 'watch'] 



as we see, we can use the pod to get information and perform actions

In [7]:
#as we see, we can use the pod to get information and perform actions
print("Information about the pod:","\n")
print("Pod namespace:",pod.namespace)
print("Pod first log:",pod.logs().split("\n")[0])

Information about the pod: 

Pod namespace: default
Pod first log: Changing ownership of /home/arkaisho to 1000:100 with options ''


## 2 making a request (create, expose and delete)
Pykube-ng also allows you to communicate with the API by sending requests with instructions to it. requests need the api object and a python dictionary with the information.

### 2.1 Creating a pod
To create a pod you can use the api already created in previous steps and a dict describing a nginx pod. Use this dict to create your pod:

In [10]:
obj = {'apiVersion': 'v1', 'kind': 'Pod', 'metadata': {'name': 'nginx', 'labels': {'app': 'nginx'}},
       'spec': {'containers': [{'image': 'nginx', 'name': 'nginx', 'ports': [{'containerPort': 8000}]}]}}
print("the object you created is equivalent to the following .yaml file:\n",yaml.dump(obj))

the object you created is equivalent to the following .yaml file:
 apiVersion: v1
kind: Pod
metadata:
  labels:
    app: nginx
  name: nginx
spec:
  containers:
  - image: nginx
    name: nginx
    ports:
    - containerPort: 8000



next you just have to send it to the right spot on the API:

In [19]:
pykube.Pod(api,obj).create()

And now, whe you run the get pod query again there is a new nginx pod running.

In [20]:
pods = pykube.Pod.objects(api)
for pod in pods:
    print(pod)

svclb-notebook-zhk94
notebook
svclb-nginx-97ffk
nginx


### 2.2 Exposing the pod
you can expose the nginx pod by creating a service with the selector {'app': 'nginx'}

In [11]:
obj = {'apiVersion': 'v1', 'kind': 'Service', 'metadata': {'name': 'nginx'}, 'spec': {'selector': {'app': 'nginx'}, 'ports': [{'protocol': 'TCP', 'port': 8000, 'targetPort': 8000}], 'type': 'LoadBalancer'}}
print("the object you created is equivalent to the following .yaml file:\n",yaml.dump(obj))
pykube.Service(api,obj).create()

the object you created is equivalent to the following .yaml file:
 apiVersion: v1
kind: Service
metadata:
  name: nginx
spec:
  ports:
  - port: 8000
    protocol: TCP
    targetPort: 8000
  selector:
    app: nginx
  type: LoadBalancer



### 2.3 Deleting the pod and the service
To delete the pod, you just have to get it with the api and call the delete function.
> The service can be deleted the same way

In [22]:
pod = pykube.Pod.objects(api).get_by_name("nginx")
svc = pykube.Service.objects(api).get_by_name("nginx")
pod.delete()
svc.delete()

When you run the get again, they're gone.

In [12]:
pods = pykube.Pod.objects(api)
for pod in pods:
    print(pod)

svclb-notebook-h7pxg
notebook-7dd7db75cd-g8h7f
